# LSTM

This first section queries the phish.net API to get a long csv of phish setlist data.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Masking
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [3]:
df = pd.read_csv('/content/allphishsets.csv')

This section explores the downloaded data and transforms the infrequently played songs into the "wildcard" song.

In [4]:
songstring = df[['showdate','set','slug']].groupby(['showdate','set'])['slug']\
                                          .apply(lambda x: '|'.join(x)).reset_index()
songstring['full'] = songstring.apply(lambda row: f"set-{row['set']}|{row['slug']}", axis=1)

songstring = songstring[['showdate','full']].groupby(['showdate'])['full']\
                                            .apply(lambda x: '|'.join(x)).reset_index()

songstring


,showdate,full
0,1985-05-03,set-1|slave-to-the-traffic-light|mikes-song|da...
1,1986-04-01,set-1|quinn-the-eskimo-the-mighty-quinn|have-m...
2,1986-10-15,set-1|alumni-blues|makisupa-policeman|skin-it-...
3,1987-03-06,set-1|funky-bitch|good-times-bad-times|corinna...
4,1987-04-29,set-1|she-caught-the-katy-and-left-me-a-mule-t...
...,...,...
1545,2023-10-10,set-1|sigma-oasis|the-9th-cube|theme-from-the-...
1546,2023-10-11,set-1|set-your-soul-free|funky-bitch|roggae|ki...
1547,2023-10-13,set-1|carini|rift|halleys-comet|ghost|albuquer...
1548,2023-10-14,set-1|runaway-jim|martian-monster|sample-in-a-...


In [ ]:
# Pre-processing: tokenizing songs

In [22]:

songs = songstring['full'].str.split('|').apply(lambda x: [song.replace('-', ' ') for song in x])
unique_songs = list(set(song for sublist in songs for song in sublist))
num_songs = len(unique_songs)

# Encode songs into numerical values
song_to_index = {song: i for i, song in enumerate(unique_songs)}
index_to_song = {i: song for i, song in enumerate(unique_songs)}

In [ ]:
# 0 list in X, 1th list in y, 2nd list in X, 3rd list in y and so on.

In [23]:


# Create input sequences and target sequences
input_sequences = []
target_sequences = []

for i in range(len(songs) - 1):
    input_seq = songs.iloc[i]
    target_seq = songs.iloc[i + 1]
    input_sequences.append([song_to_index[song] for song in input_seq])
    target_sequences.append([song_to_index[song] for song in target_seq])

# Pad sequences to make them of the same length
X = pad_sequences(input_sequences, padding='pre', truncating='pre')
y = pad_sequences(target_sequences, padding='pre', truncating='pre')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:


# Define the model
embedding_dim = 50  # Adjust as needed
model = Sequential()
model.add(Embedding(input_dim=num_songs, output_dim=embedding_dim, input_length=None))
model.add(Masking(mask_value=0.0))
model.add(LSTM(units=100,activation='tanh', kernel_initializer=he_normal, return_sequences=True))
model.add(LSTM(units=100,activation='tanh', kernel_initializer=he_normal, return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(units=num_songs, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
31/31 [==============================] - 36s 258ms/step - loss: 5.0677 - accuracy: 0.5075 - val_loss: 3.4177 - val_accuracy: 0.5519
Epoch 2/100
31/31 [==============================] - 2s 64ms/step - loss: 3.3413 - accuracy: 0.5521 - val_loss: 3.2411 - val_accuracy: 0.5519
Epoch 3/100
31/31 [==============================] - 2s 62ms/step - loss: 3.2418 - accuracy: 0.5521 - val_loss: 3.1911 - val_accuracy: 0.5519
Epoch 4/100
31/31 [==============================] - 2s 64ms/step - loss: 3.1876 - accuracy: 0.5521 - val_loss: 3.1117 - val_accuracy: 0.5519
Epoch 5/100
31/31 [==============================] - 1s 29ms/step - loss: 3.0952 - accuracy: 0.5521 - val_loss: 3.0294 - val_accuracy: 0.5519
Epoch 6/100
31/31 [==============================] - 1s 18ms/step - loss: 2.9941 - accuracy: 0.5521 - val_loss: 2.9096 - val_accuracy: 0.5519
Epoch 7/100
31/31 [==============================] - 1s 26ms/step - loss: 2.8698 - accuracy: 0.5521 - val_loss: 2.7790 - val_accuracy: 0.5519
Epoc

In [26]:
# generate predictions until num predictions or Set 1
def generate_predictions(model, seed_sequence, stop_song_index, num_predictions=10):
    predicted_sequence = seed_sequence.copy()
    predicted_set = set(predicted_sequence)

    while len(predicted_sequence) < num_predictions:
        next_song_probs = model.predict(np.array([predicted_sequence]))[0][-1]
        # Exclude songs that have already been predicted
        valid_probs = [prob if i not in predicted_set else 0 for i, prob in enumerate(next_song_probs)]
        next_song_index = np.argmax(valid_probs)

        if next_song_index == stop_song_index:
            print("Got Set 1. ")
            break

        predicted_sequence.append(next_song_index)
        predicted_set.add(next_song_index)

    return predicted_sequence


In [28]:
# test example
seed_index = 5  # test example
#seed length
seed_length = 2
seed_sequence = songstring['full'].iloc[seed_index].split('|')

# Tokenize the seed sequence
seed_sequence = [song.replace('-', ' ') for song in seed_sequence][:seed_length]
org_list =  [song.replace('-', ' ') for song in songstring['full'].iloc[seed_index].split('|')]
seed_sequence_indices = [song_to_index[song] for song in seed_sequence]

# pad/trunc seed to the required length
seed_sequence_padded = pad_sequences([seed_sequence_indices], padding='pre', truncating='pre')[0]


# stop if set 1 is encountered again
stop_song = 'set 1'
stop_song_index = song_to_index[stop_song]
# set number of predictions to the original playlist - seed length, i.e. remaining songs pending from that playlist.
num_predictions = len(org_list)- seed_length #10

predicted_sequence = generate_predictions(model, seed_sequence_padded.tolist(), stop_song_index, num_predictions)

# numerical predictions back to song names
predicted_songs = [index_to_song[i] for i in predicted_sequence]

# Print the seed sequence and predicted sequence
print('\norg_list')
print(org_list)
print("\nSeed Sequence:")
print(seed_sequence)
print("\nPredicted Sequence:")
print(predicted_songs[seed_length:])



1/1 [==============================] - 0s 24ms/step

org_list
['set 1', 'you enjoy myself', 'lushington', 'possum', 'slave to the traffic light', 'sneakin sally through the alley', 'clod', 'peaches en regalia', 'the man who stepped into yesterday', 'avenu malkenu', 'the man who stepped into yesterday', 'makisupa policeman', 'ya mar', 'set 2', 'divided sky', 'funky bitch', 'harpua', 'bundle of joy', 'harpua', 'fluffhead', 'good times bad times', 'set e', 'golgi apparatus', 'corinna', 'letter to jimmy page']

Seed Sequence:
['set 1', 'you enjoy myself']

Predicted Sequence:
['family picture', 'alumni blues', 'golgi apparatus', 'wilson', 'possum', 'letter to jimmy page', 'chalk dust torture', 'harpua', 'divided sky', 'runaway jim', 'suzy greenberg', 'foam', 'llama', 'the landlady', 'reba', 'buried alive', 'guelah papyrus', 'poor heart', 'the man who stepped into yesterday', 'bouncing around the room', 'acdc bag']


In [29]:
# Assuming org_list, seed_sequence, stopword_list, and predicted_sequence are lists of songs

stopword_list = {'set 1', 'set 2', 'set 3', 'set 4', 'set 5', 'set 6', 'set e'}


# Convert the lists to sets
org_set = set(org_list)
seed_set = set(seed_sequence)

# Find the songs in org_list but not in seed_sequence
not_in_seed_sequence = org_set - seed_set

# Remove songs in stopword_list
filtered_songs = [song.strip() for song in not_in_seed_sequence if song not in stopword_list]

# Remove songs in stopword_list from predicted_sequence
predicted_sequence = [song.strip() for song in predicted_songs[seed_length:] if song not in stopword_list]

# Count the number of matching songs
matching_songs = [song for song in filtered_songs if song in predicted_sequence]
num_matching_songs = len(matching_songs)

# Calculate the percentage of matching songs
percentage_matching = (num_matching_songs / len(filtered_songs)) * 100

print("Matching Songs in Predicted Sequence:")
print(matching_songs)
print("Number of Matching Songs:", num_matching_songs)
print("Percentage of Matching Songs:", percentage_matching)


Matching Songs in Predicted Sequence:
['the man who stepped into yesterday', 'divided sky', 'harpua', 'golgi apparatus', 'possum', 'letter to jimmy page']
Number of Matching Songs: 6
Percentage of Matching Songs: 31.57894736842105


In [ ]:
## Splitting by date

In [48]:
split_date = '2010-12-31'

# Split the data
train_data = songstring[songstring['showdate'] < split_date]
test_data = songstring[songstring['showdate'] >= split_date]

# Tokenize songs in the training and testing data
train_songs = train_data['full'].str.split('|').apply(lambda x: [song.replace('-', ' ') for song in x])
test_songs = test_data['full'].str.split('|').apply(lambda x: [song.replace('-', ' ') for song in x])

# Create unique songs and encode into numerical values
all_songs = list(set(song for sublist in train_songs for song in sublist))
num_songs = len(all_songs)

song_to_index = {song: i for i, song in enumerate(all_songs)}
index_to_song = {i: song for i, song in enumerate(all_songs)}

# Encode sequences for training data
train_input_sequences = []
train_target_sequences = []

for i in range(len(train_songs) - 1):
    input_seq = train_songs.iloc[i]
    target_seq = train_songs.iloc[i + 1]
    train_input_sequences.append([song_to_index[song] for song in input_seq])
    train_target_sequences.append([song_to_index[song] for song in target_seq])

# Pad sequences for training and testing data
X_train = pad_sequences(train_input_sequences, padding='pre', truncating='pre')
y_train = pad_sequences(train_target_sequences, padding='pre', truncating='pre')
# Encode sequences for testing data
test_input_sequences = []
test_target_sequences = []

for i in range(len(test_songs) - 1):
    input_seq = test_songs.iloc[i]
    target_seq = test_songs.iloc[i + 1]

    # Handle missing songs in the dictionary
    input_sequence_indices = [song_to_index.get(song, -1) for song in input_seq]
    target_sequence_indices = [song_to_index.get(song, -1) for song in target_seq]

    # Filter out songs with index -1 (not found in the dictionary)
    input_sequence_indices = [index for index in input_sequence_indices if index != -1]
    target_sequence_indices = [index for index in target_sequence_indices if index != -1]

    test_input_sequences.append(input_sequence_indices)
    test_target_sequences.append(target_sequence_indices)

# Pad sequences for testing data
X_test = pad_sequences(test_input_sequences, padding='pre', truncating='pre')
y_test = pad_sequences(test_target_sequences, padding='pre', truncating='pre')

In [49]:


# model
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim=num_songs, output_dim=embedding_dim, input_length=None))
model.add(Masking(mask_value=0.0))
model.add(LSTM(units=100,activation='tanh', kernel_initializer=he_normal, return_sequences=True))
model.add(LSTM(units=100,activation='tanh', kernel_initializer=he_normal, return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(units=num_songs, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
34/34 [==============================] - 16s 202ms/step - loss: 4.7789 - accuracy: 0.5087 - val_loss: 3.4108 - val_accuracy: 0.5672
Epoch 2/100
34/34 [==============================] - 3s 98ms/step - loss: 3.2688 - accuracy: 0.5450 - val_loss: 3.1939 - val_accuracy: 0.5672
Epoch 3/100
34/34 [==============================] - 2s 51ms/step - loss: 3.0967 - accuracy: 0.5450 - val_loss: 3.0293 - val_accuracy: 0.5672
Epoch 4/100
34/34 [==============================] - 2s 48ms/step - loss: 2.9209 - accuracy: 0.5450 - val_loss: 2.8345 - val_accuracy: 0.5672
Epoch 5/100
34/34 [==============================] - 1s 32ms/step - loss: 2.7518 - accuracy: 0.5450 - val_loss: 2.7066 - val_accuracy: 0.5663
Epoch 6/100
34/34 [==============================] - 1s 18ms/step - loss: 2.6323 - accuracy: 0.5456 - val_loss: 2.6372 - val_accuracy: 0.5663
Epoch 7/100
34/34 [==============================] - 1s 19ms/step - loss: 2.5709 - accuracy: 0.5491 - val_loss: 2.5851 - val_accuracy: 0.5663
Epoc

In [56]:
# test example
seed_index = 1500 # test example
#seed length
seed_length =2
seed_sequence = songstring['full'].iloc[seed_index].split('|')

# Tokenize the seed sequence
seed_sequence = [song.replace('-', ' ') for song in seed_sequence][:seed_length]
org_list =  [song.replace('-', ' ') for song in songstring['full'].iloc[seed_index].split('|')]
seed_sequence_indices = [song_to_index[song] for song in seed_sequence]

# pad/trunc seed to the required length
seed_sequence_padded = pad_sequences([seed_sequence_indices], padding='pre', truncating='pre')[0]


# stop if set 1 is encountered again
stop_song = 'set 1'
stop_song_index = song_to_index[stop_song]
# set number of predictions to the original playlist - seed length, i.e. remaining songs pending from that playlist.
num_predictions = len(org_list)- seed_length #10

predicted_sequence = generate_predictions(model, seed_sequence_padded.tolist(), stop_song_index, num_predictions)

# numerical predictions back to song names
predicted_songs = [index_to_song[i] for i in predicted_sequence]

# Print the seed sequence and predicted sequence
print('\norg_list')
print(org_list)
print("\nSeed Sequence:")
print(seed_sequence)
print("\nPredicted Sequence:")
print(predicted_songs[seed_length:])


1/1 [==============================] - 0s 29ms/step

org_list
['set 1', 'the landlady', 'runaway jim', 'divided sky', '46 days', 'stealing time from the faulty plan', 'sugar shack', 'shade', 'halfway to the moon', 'everythings right', 'set 2', 'energy', 'gotta jibboo', 'soul planet', 'rift', 'reba', 'martian monster', 'broken heart attack', 'hold your head up', 'martian monster', 'possum', 'slave to the traffic light', 'set e', 'waste', 'first tube']

Seed Sequence:
['set 1', 'the landlady']

Predicted Sequence:
['night nurse', 'feel the heat', 'the other one', 'wild child', 'let it loose', 'family picture', 'blackberry blossom', 'golgi apparatus', 'alumni blues', 'divided sky', 'letter to jimmy page', 'the man who stepped into yesterday', 'possum', 'buried alive', 'suzy greenberg', 'chalk dust torture', 'wilson', 'reba', 'runaway jim', 'foam', 'llama']


In [57]:
# Assuming org_list, seed_sequence, stopword_list, and predicted_sequence are lists of songs

stopword_list = {'set 1', 'set 2', 'set 3', 'set 4', 'set 5', 'set 6', 'set e'}


# Convert the lists to sets
org_set = set(org_list)
seed_set = set(seed_sequence)

# Find the songs in org_list but not in seed_sequence
not_in_seed_sequence = org_set - seed_set

# Remove songs in stopword_list
filtered_songs = [song.strip() for song in not_in_seed_sequence if song not in stopword_list]

# Remove songs in stopword_list from predicted_sequence
predicted_sequence = [song.strip() for song in predicted_songs[seed_length:] if song not in stopword_list]

# Count the number of matching songs
matching_songs = [song for song in filtered_songs if song in predicted_sequence]
num_matching_songs = len(matching_songs)

# Calculate the percentage of matching songs
percentage_matching = (num_matching_songs / len(filtered_songs)) * 100

print("Matching Songs in Predicted Sequence:")
print(matching_songs)
print("Number of Matching Songs:", num_matching_songs)
print("Percentage of Matching Songs:", percentage_matching)


Matching Songs in Predicted Sequence:
['runaway jim', 'divided sky', 'possum', 'reba']
Number of Matching Songs: 4
Percentage of Matching Songs: 20.0


In [59]:

songs = songstring['full'].str.split('|').apply(lambda x: [song.replace('-', ' ') for song in x])
unique_songs = list(set(song for sublist in songs for song in sublist))
num_songs = len(unique_songs)

# Encode songs into numerical values
song_to_index = {song: i for i, song in enumerate(unique_songs)}
index_to_song = {i: song for i, song in enumerate(unique_songs)}

In [60]:
X_train_all = []
y_train_all = []

for song_list in songs:
    sequences = []
    for i in range(1, len(song_list)):
        sequence = song_list[:i + 1]  # Take the first i+1 songs
        sequences.append(sequence)

    # Create input (X_train) and target (y_train) sequences for the current list
    X_train = [seq[:-1] for seq in sequences]  # All songs except the last one
    y_train = [seq[-1] for seq in sequences]  # The last song

    # Convert song titles to corresponding indices using the mapping
    X_train = [[song_to_index[song] for song in seq] for seq in X_train]
    y_train = [song_to_index[song] for song in y_train]

    # Append the sequences for the current list to the overall X_train_all and y_train_all
    X_train_all.extend(X_train)
    y_train_all.extend(y_train)

In [61]:
X_train_all[0],X_train_all[1],X_train_all[2],X_train_all[3],X_train_all[4]

([279],
 [279, 82],
 [279, 82, 179],
 [279, 82, 179, 342],
 [279, 82, 179, 342, 464])

In [64]:

# Pad sequences to make them of the same length
X_train_all_padded = pad_sequences(X_train_all, padding='pre', value=0.0)  # Use a value that won't interfere with your song indices

# Convert lists to numpy arrays
X_train_all_padded = np.array(X_train_all_padded)
y_train_all = np.array(y_train_all)

In [65]:
embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=num_songs, output_dim=embedding_dim, input_length=None))
model.add(Masking(mask_value=0.0))
model.add(LSTM(units=100,activation='tanh', kernel_initializer=he_normal, return_sequences=True))
model.add(LSTM(units=100,activation='tanh', kernel_initializer=he_normal))
model.add(Dropout(0.5))
model.add(Dense(units=num_songs, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [66]:
from keras.callbacks import EarlyStopping
es=  EarlyStopping(monitor='val_loss',patience=10)
model.fit(X_train_all_padded, y_train_all, epochs=100, batch_size=32, validation_split=0.3,callbacks = [es])

Epoch 1/100
804/804 [==============================] - 33s 31ms/step - loss: 5.1820 - accuracy: 0.0678 - val_loss: 6.0132 - val_accuracy: 0.0897
Epoch 2/100
804/804 [==============================] - 14s 18ms/step - loss: 4.7474 - accuracy: 0.1001 - val_loss: 5.9906 - val_accuracy: 0.1018
Epoch 3/100
804/804 [==============================] - 14s 18ms/step - loss: 4.5138 - accuracy: 0.1238 - val_loss: 6.0566 - val_accuracy: 0.1181
Epoch 4/100
804/804 [==============================] - 15s 18ms/step - loss: 4.3661 - accuracy: 0.1447 - val_loss: 6.1624 - val_accuracy: 0.1146
Epoch 5/100
804/804 [==============================] - 14s 17ms/step - loss: 4.2482 - accuracy: 0.1574 - val_loss: 6.1405 - val_accuracy: 0.1205
Epoch 6/100
804/804 [==============================] - 14s 18ms/step - loss: 4.1536 - accuracy: 0.1679 - val_loss: 6.2686 - val_accuracy: 0.1227
Epoch 7/100
804/804 [==============================] - 13s 17ms/step - loss: 4.0603 - accuracy: 0.1776 - val_loss: 6.2354 - val_ac